In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds

데이터 셋 로드 (TFDS에서 가져온 EMNIST)

In [10]:
(etfds_train, etfds_test), etfds_info = tfds.load("emnist", split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)
print(etfds_info)

def normalize_img_ascii_lable(image, label):
    ascii_Lb = label
    if ascii_Lb >= 10:
        ascii_Lb += 8
    elif ascii_Lb >= 43:
        ascii_Lb += 6
    ascii_Lb += 15
    return tf.cast(image, tf.float32) / 255., ascii_Lb

etfds_train = etfds_train.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_train = etfds_train.cache()
etfds_train = etfds_train.shuffle(etfds_info.splits['train'].num_examples)
etfds_train = etfds_train.batch(128)
etfds_train = etfds_train.prefetch(tf.data.experimental.AUTOTUNE)

etfds_test = etfds_test.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_test = etfds_test.batch(128)
etfds_test = etfds_test.cache()
etfds_test = etfds_test.prefetch(tf.data.experimental.AUTOTUNE)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 28.73 url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 19.68 url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Dataset emnist downloaded and prepared to C:\Users\ssm01\tensorflow_datasets\emnist\byclass\3.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='emnist',
    full_name='emnist/byclass/3.0.0',
    description="""
    The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19 and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset.
    
    Note: Like the original EMNIST data, images provided here are inverted horizontally and rotated 90 anti-clockwise. You can use `tf.transpose` within `ds.map` to convert the images to a human-friendlier format.
    """,
    config_description="""
    EMNIST ByClass
    """,
    homepage='https://www.nist.gov/itl/products-and-services/emnist-dataset',
    data_path='C:\\Users\\ssm01\\tensorflow_datasets\\emnist\\byclass\\3.0.0',
    file_format=tfrecord,
    download_size=535.73 MiB,
    dataset_size=349.16 MiB,
    features=FeaturesDic

직접 만든 데이터 셋 로드

In [14]:
import pathlib

data_dir = pathlib.Path('..\dataset')

trainFG = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(28,28),
    batch_size=16,
    label_mode='int'
)

print(trainFG.class_names)

testFG = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(28,28),
    batch_size=16,
    label_mode='int'
)

Found 1520 files belonging to 95 classes.
Using 1216 files for training.
['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94']
Found 1520 files belonging to 95 classes.
Using 304 files for validation.


모델 세팅

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(95, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 95)                48735     
                                                                 
Total params: 450655 (1.72 MB)
Trainable params: 450655 (1.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


훈련시키기

In [13]:
model.fit(trainFG, epochs=5)
model.fit(etfds_train, epochs=5)


Epoch 1/5


ValueError: in user code:

    File "c:\Python311\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Python311\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Python311\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Python311\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "c:\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Python311\Lib\site-packages\keras\src\engine\input_spec.py", line 280, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling layer 'sequential' (t

모델 저장하기

In [ ]:
model.save('spemnist.h5')

확인하기

In [ ]:
model.evaluate(etfds_test, verbose=2)
model.evaluate(testFG, verbose=2)